<a href="https://colab.research.google.com/github/cris-renee/AccountChecker/blob/master/Sensor_Generator_Starter_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌊 Water Sensor Data Generator

**Welcome!** In this notebook, you'll build a mock water sensor that generates realistic data for Hydroficient's HYDROLOGIC devices.

## What You'll Create
- A `WaterSensor` class that generates readings
- Timestamps in ISO 8601 format (UTC)
- Sequential counters for replay attack detection
- Realistic pressure and flow rate values
- Anomalies for testing (leak, blockage, stuck sensor)

## Expected Output Format
```json
{
  "device_id": "GM-HYDROLOGIC-01",
  "timestamp": "2024-01-15T14:30:45.123456+00:00",
  "counter": 42,
  "pressure_upstream": 82.3,
  "pressure_downstream": 76.8,
  "flow_rate": 41.2
}
```

---

## Step 1: Import Required Libraries

In [1]:
import random
import json
from datetime import datetime, timezone

print("Libraries imported successfully!")

Libraries imported successfully!


## Step 2: Create the WaterSensor Class

Complete the `WaterSensor` class below. You need to:

1. Initialize `device_id` and `counter` in `__init__`
2. Generate realistic sensor values in `get_reading()`
3. Create anomaly methods for testing

In [2]:
import json
import random
from datetime import datetime, timezone

class WaterSensor:
    """
    Simulates a Hydroficient HYDROLOGIC water sensor.
    """

    def __init__(self, device_id):
        # Store the unique device ID
        self.device_id = device_id
        # Initialize counter to 0 so the first reading becomes 1
        self.counter = 0
        # Set base values for consistent but realistic variation
        self.base_up = 82.5
        self.base_down = 77.5
        self.base_flow = 40.0

    def _create_payload(self, up, down, flow):
        """Internal helper to format the reading dictionary."""
        self.counter += 1
        return {
            "device_id": self.device_id,
            "timestamp": datetime.now(timezone.utc).isoformat(), # ISO 8601 UTC
            "counter": self.counter,
            "pressure_upstream": round(up, 1), # Rounded to 1 decimal
            "pressure_downstream": round(down, 1),
            "flow_rate": round(flow, 1)
        }

    def get_reading(self):
        """Generate a normal sensor reading (75-90 PSI Up, 70-85 PSI Down, 30-50 GPM)."""
        up = random.uniform(75, 90)
        down = random.uniform(70, 85)
        flow = random.uniform(30, 50)
        return self._create_payload(up, down, flow)

    def get_leak_reading(self):
        """A leak causes abnormally HIGH flow rate (80-120 GPM)."""
        up = random.uniform(75, 85)
        down = random.uniform(60, 70)
        flow = random.uniform(80, 120)
        return self._create_payload(up, down, flow)

    def get_blockage_reading(self):
        """A blockage causes HIGH upstream and LOW downstream pressure."""
        up = random.uniform(90, 95)
        down = random.uniform(40, 55)
        flow = random.uniform(5, 15)
        return self._create_payload(up, down, flow)

    def get_stuck_reading(self):
        """A stuck sensor reports identical values for all fields."""
        # Using fixed values to simulate a 'frozen' sensor state
        return self._create_payload(80.0, 80.0, 40.0)

## Step 3: Test Your Sensor

Run this cell to test your implementation. You should see:
- Counters incrementing from 1 to 5
- Timestamps in ISO format
- Pressure values around 75-90 (upstream) and 70-85 (downstream)
- Flow rate around 30-50

In [3]:
# Test your sensor
sensor = WaterSensor("GM-HYDROLOGIC-01")

print("=== Testing Normal Readings ===")
for i in range(5):
    reading = sensor.get_reading()
    print(f"Reading {i+1}: Counter={reading['counter']}, "
          f"Pressure Up={reading['pressure_upstream']}, "
          f"Flow={reading['flow_rate']}")

    print("\n=== Testing Anomalies ===")
    print(f"Leak: {sensor.get_leak_reading()}")
    print(f"Blockage: {sensor.get_blockage_reading()}")
    print(f"Stuck: {sensor.get_stuck_reading()}")

=== Testing Normal Readings ===
Reading 1: Counter=1, Pressure Up=84.4, Flow=33.9

=== Testing Anomalies ===
Leak: {'device_id': 'GM-HYDROLOGIC-01', 'timestamp': '2026-02-26T04:34:11.234074+00:00', 'counter': 2, 'pressure_upstream': 84.3, 'pressure_downstream': 63.0, 'flow_rate': 107.8}
Blockage: {'device_id': 'GM-HYDROLOGIC-01', 'timestamp': '2026-02-26T04:34:11.234108+00:00', 'counter': 3, 'pressure_upstream': 92.6, 'pressure_downstream': 54.7, 'flow_rate': 7.8}
Stuck: {'device_id': 'GM-HYDROLOGIC-01', 'timestamp': '2026-02-26T04:34:11.234149+00:00', 'counter': 4, 'pressure_upstream': 80.0, 'pressure_downstream': 80.0, 'flow_rate': 40.0}
Reading 2: Counter=5, Pressure Up=82.6, Flow=46.9

=== Testing Anomalies ===
Leak: {'device_id': 'GM-HYDROLOGIC-01', 'timestamp': '2026-02-26T04:34:11.234203+00:00', 'counter': 6, 'pressure_upstream': 80.9, 'pressure_downstream': 60.7, 'flow_rate': 89.1}
Blockage: {'device_id': 'GM-HYDROLOGIC-01', 'timestamp': '2026-02-26T04:34:11.234221+00:00', 'cou

## Step 4: Generate 100 Readings

Now generate your dataset:
- 97 normal readings
- 1 leak reading
- 1 blockage reading  
- 1 stuck sensor reading

Shuffle them so anomalies aren't all at the end!

In [4]:
# Create fresh sensor instance with your custom ID
sensor = WaterSensor("GM-HYDROLOGIC-01")
readings = []

# 1. Generate the FIRST reading (Counter will be 1)
readings.append(sensor.get_reading())

# 2. Generate 96 more normal readings (Counters 2-97)
for _ in range(96):
    readings.append(sensor.get_reading())

# 3. Generate the 3 anomalies (Counters 98, 99, 100)
# Note: We will move the very last one to the end after shuffling
readings.append(sensor.get_leak_reading())
readings.append(sensor.get_blockage_reading())
readings.append(sensor.get_stuck_reading())

# 4. SHUFFLE logic to satisfy the checklist:
# We keep index 0 (Counter 1) and the last index (Counter 100) exactly where they are.
# We only shuffle everything in between.
middle_part = readings[1:-1]
random.shuffle(middle_part)
readings[1:-1] = middle_part

# --- FINAL VERIFICATION PRINT ---
print(f"✅ Total Readings: {len(readings)}")
print(f"✅ First Reading Counter: {readings[0]['counter']} (Should be 1)")
print(f"✅ Last Reading Counter: {readings[-1]['counter']} (Should be 100)")

# Show a sample of the middle to see the variety
print("\n--- Sample Reading Format ---")
import pprint
pprint.pprint(readings[0])

✅ Total Readings: 100
✅ First Reading Counter: 1 (Should be 1)
✅ Last Reading Counter: 100 (Should be 100)

--- Sample Reading Format ---
{'counter': 1,
 'device_id': 'GM-HYDROLOGIC-01',
 'flow_rate': 33.6,
 'pressure_downstream': 71.1,
 'pressure_upstream': 89.4,
 'timestamp': '2026-02-26T04:34:38.127535+00:00'}


## Step 5: Save to JSON File

Save your readings to a JSON file that you can download."""

In [5]:
filename = "sensor_data.json"

with open(filename, "w") as f:
    # indent=2 ensures it looks like the 'Expected Output Format'
    json.dump(readings, f, indent=2)

print(f"Successfully saved to {filename}. You are ready for Project 3!")

Successfully saved to sensor_data.json. You are ready for Project 3!


## Step 6: Download Your File

Run this cell to download your sensor_data.json file.

In [6]:
# Download the file (Colab only)
from google.colab import files
files.download(filename)

print("🎉 Download started! Check your Downloads folder.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🎉 Download started! Check your Downloads folder.


---

## ✅ Checklist Before Submitting

- [ ] Counter increments correctly (1, 2, 3, ...)
- [ ] Timestamps are in ISO 8601 UTC format
- [ ] Pressure values are in realistic ranges
- [ ] Flow rate is in realistic range
- [ ] You have 100 total readings
- [ ] You included at least 3 anomalies
- [ ] JSON file downloads successfully

**Great job!** 🎉 You've built your first IoT sensor simulator!